In [59]:
import pandas as pd
kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="/home/uc4ddc6536e59d9d8f8f5069efdb4e25/kaggle_hms/kaggle/temp1"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)


In [60]:
import subprocess
cp_path=f'{kaggle_temp}/f_data_cp'
cp_path1=f'{cp_path}/f_data_cp#1.csv'
cp_size=1000
def checkpoint_data(f_data):
    try:
        display("checkpoint data=",f_data)
        cps=str(subprocess.check_output(f"ls '{cp_path}' | grep 'f_data_cp#'",shell=True,text=True))
        cps=cps.strip('\n').split('\n')
        cps.sort(key=lambda x : int(x.split('#')[1].split('.')[0]))
        print(cps)
           
        # if len(cps)==1:
        # for appending checkpoints
        if True:
            cp=cps[-1]
            cp_num=int(cp.split('#')[1].split('.')[0])
            new_cp_path=cp_path1.replace('#1',f'#{cp_num+1}')
            pi=max(f_data.index[-1]-cp_size,f_data.index[0])
            print(f'writing from {pi} to {f_data.index[-1]}')
            display(f_data.loc[pi:])
            f_data.loc[pi:].to_csv(new_cp_path,index=True)
            
        # else:
        #     cp=cps[1]
        #     cp_num=int(cp.split('#')[1].split('.')[0])
        #     new_cp_path=cp_path1.replace('#1',f'#{cp_num+1}')
        #     f_data.to_csv(new_cp_path,index=True)
            # print(subprocess.check_output(f"rm '{cp_path}/{cps[0]}'",shell=True,text=True))
    except Exception as e:
        print(e)
        print("creating 1st cp")
        new_cp_path=cp_path1
        f_data.to_csv(new_cp_path,index=True)


def load_checkpoint_data(full_data=False):
    try:
        cps=subprocess.check_output(f"ls {cp_path} | grep 'f_data_cp#'",shell=True,text=True)
        cps=cps.strip('\n').split('\n')
        cps.sort(key=lambda x : int(x.split('#')[1].split('.')[0]))
        if full_data==False:
            cp=cps[-1]
            new_cp_path=f'{cp_path}/{cp}'
            cpdf=pd.read_csv(new_cp_path,index_col=0)
            print("----checkpoint loaded with len ------------",len(cpdf))
            return cpdf
        
        if full_data==True:
            f_data=pd.DataFrame()
            for cp in cps:
                # cp_num=int(cp.split('#')[1].split('.')[0])
                new_cp_path=f'{cp_path}/{cp}'
                cpdf=pd.read_csv(new_cp_path,index_col=0)
                f_data=pd.concat([f_data,cpdf],axis=0,ignore_index=False)
                
            print("----checkpoint loaded with len ------------",len(f_data))
            return f_data



    
        

        # if len(cps)==1:
        #     new_cp_path=f"{cp_path}/{cps[0]}"
        # else:
        #     new_cp_path=f"{cp_path}/{cps[1]}"
        # f_data=pd.read_csv(new_cp_path,index_col=0)
        # return f_data
    except Exception as e:
        print(e)
        return pd.DataFrame([])


In [61]:
# plen=0.001
plen=1
# format test data
train_data=pd.read_csv(f"{kaggle_input}/train.csv")
test_data=pd.read_csv(f"{kaggle_input}/test.csv")
f_names=['f_train.csv','f_test.csv']
fi=0

f_data=train_data.copy() if fi==0 else test_data.copy()
if fi==0:
    f_data=f_data.loc[:len(f_data)*plen,:]

# embedding averages for spectogram
if fi==0:
    f_data_cp=load_checkpoint_data()
    start_cp=start_cpi=len(f_data_cp)
    savg_cols=[]
    tdf_avgs=pd.DataFrame()
    if len(f_data_cp)!=0:
        savgs_index=[i+1 for i in range(len(f_data_cp.columns)) if f_data_cp.columns[i]=='other_vote']
        tdf_avgs=f_data_cp.iloc[:,savgs_index[0]:]
        print("after splitting tdf_avgs and f_data_cp index=",tdf_avgs.index[0],f_data_cp.index[0])
        savg_cols=tdf_avgs.columns
else:
    start_cp=0
    savg_cols=[]
    tdf_avgs=pd.DataFrame([])
for i in range(start_cp,len(f_data)):
    # checkpointing data
    if fi==0 and i%cp_size==0 and i!=start_cp:
        # t_df=pd.concat([f_data.iloc[start_cpi:i],tdf_avgs.iloc[max(i-start_cp-cp_size,0):].set_axis([i for i in range(start_cpi,i)])],axis=1)
        t_df=pd.concat([f_data.iloc[start_cpi:i],tdf_avgs.iloc[start_cpi:i]],join='inner',axis=1)
        # t_df=t_df.set_axis([i for i in range(f_data_cp.index[-1]+1,f_data_cp.index[-1]+1+len(t_df))])
        f_data_cp=pd.concat([f_data_cp,t_df],axis=0)
        f_data_cp=f_data_cp.set_axis([i for i in range(f_data_cp.index[0],f_data_cp.index[0]+len(f_data_cp))])
        if i!=start_cp:
            checkpoint_data(f_data_cp)
        start_cpi=i

    spect_id=f_data.loc[i,'spectrogram_id']
    # print(spect_id)
    if False and fi==0 and f_data.loc[i,'spectrogram_sub_id']!=0 and f_data.loc[i,'spectogram_id']==f_data.loc[i-1,'spectogram_id'] and tdf_avgs.iloc[i-1,0].notnull():
        rdf_avgs=tdf_avgs.loc[i-1:i-1,savg_cols].copy()
    else:
        spect_data=pd.read_parquet(f"{kaggle_input}/{'train' if fi==0 else 'test'}_spectrograms/{spect_id}.parquet")
        spect_cols=spect_data.columns
        spect_cols=[spect_cols[i] for i in range(len(spect_cols)) if i%5==0 and i!=0]
        # print(spect_cols)
        # print(len(spect_cols))
        rdf_avgs=pd.DataFrame()
        for spect_col in spect_cols:
            avgi=5
            avgs=[round(sum(spect_data.loc[i:i+avgi,spect_col])/avgi,3) for i in range(0,min(len(spect_data),300),avgi)]
            # print(avgs)
            # print(len(avgs))
            df_avgs=pd.DataFrame([avgs],columns=[f'{spect_col}savg{t}' for t in range(len(avgs)) ])
            rdf_avgs=pd.concat([rdf_avgs,df_avgs],axis=1)
    # display("rdf=",rdf_avgs)
    tdf_avgs=pd.concat([tdf_avgs,rdf_avgs],axis=0,ignore_index=True)
    savg_cols=tdf_avgs.columns.tolist()
    # display("tdf=",tdf_avgs)
f_data=pd.concat([f_data,tdf_avgs],axis=1)   
        

# print(len(f_data.columns))
f_data[:3]



----checkpoint loaded with len ------------ 51
after splitting tdf_avgs and f_data_cp index= 89999 89999


In [ ]:
f_data_cp.loc[90448:]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
90448,3548348860,34,144.0,1841034439,337,...,0.748,623.332004,38.180001,0.188,0.116
90449,3548348860,35,148.0,1841034439,338,...,0.748,623.332004,38.180001,0.188,0.116
90450,2882017352,2,8.0,1910466,2,...,0.078,0.086000,0.074000,0.116,0.046
90451,2882017352,3,10.0,1910466,3,...,0.078,0.086000,0.074000,0.116,0.046
90452,2882017352,4,14.0,1910466,4,...,0.078,0.086000,0.074000,0.116,0.046
...,...,...,...,...,...,...,...,...,...,...,...
90494,893864755,5,30.0,4367732,5,...,0.020,0.018000,0.020000,0.016,0.020
90495,3669581453,0,0.0,4546138,0,...,1.326,0.920000,2.274000,1.590,1.542
90496,3669581453,1,4.0,4546138,1,...,1.326,0.920000,2.274000,1.590,1.542
90497,3669581453,2,6.0,4546138,2,...,1.326,0.920000,2.274000,1.590,1.542


In [ ]:
# f_data_cp[:203]
tdf_avgs[7445:7455]

In [ ]:
if fi==0:
    f_data=load_checkpoint_data()
# keeping cols
print(savg_cols)
vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
# savg_cols=[f'savg{i}' for i in range(60)]
f_data=f_data[[col for col in vote_cols+savg_cols] if fi==0 else [col for col in ['eeg_id']+savg_cols]]
if fi==0:
    # converting votes to probs
    for i in range(len(f_data)):
        sum_votes=sum(f_data.loc[i,vote_cols])
        f_data.loc[i,vote_cols]=f_data.loc[i,vote_cols].apply(lambda x: (x/sum_votes) )

# saving to csv
f_data.to_csv(f'{kaggle_temp}/{f_names[fi]}',index=False)

In [ ]:
t_df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
100,2882017352,2,8.0,1910466,2,...,0.078,0.086,0.074,0.116,0.046
101,2882017352,3,10.0,1910466,3,...,0.078,0.086,0.074,0.116,0.046
102,2882017352,4,14.0,1910466,4,...,0.078,0.086,0.074,0.116,0.046
103,2578018731,0,0.0,2207717,0,...,0.126,0.184,0.212,0.160,0.146
104,2578018731,1,20.0,2207717,1,...,0.126,0.184,0.212,0.160,0.146
...,...,...,...,...,...,...,...,...,...,...,...
144,893864755,5,30.0,4367732,5,...,0.020,0.018,0.020,0.016,0.020
145,3669581453,0,0.0,4546138,0,...,1.326,0.920,2.274,1.590,1.542
146,3669581453,1,4.0,4546138,1,...,1.326,0.920,2.274,1.590,1.542
147,3669581453,2,6.0,4546138,2,...,1.326,0.920,2.274,1.590,1.542
